## 栈和列表的应用：迷宫的求解

* 迷宫用0/1来表示
* 对于其中的某一个格子，如果它的方位是`(i,j)`，则他四周的方位则表示为这个二元组分别加上dirs：

In [2]:
dirs = [(0,1),(1,0),(0,-1),(-1,0)]

为了使算法的描述更加方便，定义两个辅助函数：

In [1]:
def mark(maze, pos): # 给迷宫maze的位置pos标识“到过了”
    maze[pos[0]][pos[1]] = 2

def passable(maze, pos): # 检查迷宫maze的位置pos是否可行
    return maze[pos[0]][pos[1]] == 0

### 迷宫的递归求解
有关迷宫求解问题的递归观点如下：
* 每一时刻应当有一个当前位置，开始时这个位置是迷宫入口
* 如果当前位置是出口，问题已解决
* 否则，如果从当前位置已经无路可走，表示探查失败，回退一步。
* 取一个可行相邻位置，用同样的方法探查，如果从那里可以找到通往出口的路径，那么从当前位置到出口的路径就找到了。


In [4]:
# 递归核心函数的实现
def find_path(maze, pos, end):
    mark(maze, pos)
    if pos == end: # 已到达出口
        print(pos, end = " ")
        return True
    for i in range(4):
        nextp = pos[0] + dirs[i][0], pos[1] + dirs[i][1]
        if passable(maze, nextp):
            if find_path(maze, nextp, end):
                print(pos, end = " ")
                return True
    return False

In [5]:
maze = [[1,1,1,1,1],[1,0,0,1,1],[1,1,0,0,1],[1,1,1,0,1],[1,1,1,1,1]]
pos = (1,1)
end = (3,3)

In [7]:
find_path(maze, pos, end)

(3, 3) (2, 3) (2, 2) (1, 2) (1, 1) 

True

### 栈和回溯法
如果不用递归技术求解迷宫问题，其中一种方法称为**回溯法**，这种算法在工作中执行两种基本动作：前进和后退。
* 前进
  * 条件：当前位置存在尚未探查的四邻位置
  * 操作：选定下一个位置并向前探查。如果还存在其他没有被探查的分支，就记录相关信息以便将来使用。
* 后退
  * 条件：遇到死路
  * 操作：退回最近记录的分支点，检查那里是否存在未探查分支。如果有就取一个为探查的位置作为当前位置并前进，没有就将其删除并继续回溯。
* 已穷尽所有可能：不能找到出口时以失败结束。

易见，这里的分支的删除和使用具有**栈**的性质。所以可以用一个栈作为辅助结构来保存工作中发现的回溯点。

**入栈的元素：**途径每一个位置时，先检查那里的情况，在栈中只保存还存在其他未探查方向的位置。这种保存方式节省栈的空间

## 算法的实现
根据迷宫的表示以及回溯后继续搜索的需要，存入栈的是序队（pos, nxt）,pos表示位置，nxt则表示回溯到该位置的下一探索方向，四个方向用编码（0,1,2,3）来表示

In [54]:
# 先定义一个辅助函数print_path,它应该把栈中的位置和当前位置加end输出。
def print_path(end, pos, st):
    while not st.is_empty():
        print(st.pop()[0], end = " ")
    print(pos, end)

In [55]:
# 补充一下SStack的定义
class SStack():
    def __init__(self):
        self._elems = []
    
    def is_empty(self):
        return self._elems == []

    def top(self):
        if self._elems == []:
            raise StackUnderflow("in SStack.top()")
        return self.elem[-1] # 后进先出，得到最后一个元素
    
    def push(self, elem):
        self._elems.append(elem)
    
    def pop(self):
        if self._elems == []:
            raise StackUnderflow("in SStack.pop()")
        return self._elems.pop() # 以列表形式弹出
    
    def printall(self):
        print(self._elems)

In [56]:
def maze_solve(maze, start, end):
    if start == end:
        print(start)
        return
    st = SStack() # 创建栈对象
    mark(maze, start)
    st.push((start, 0)) # 入口和方向0入栈
    while not st.is_empty(): # 走不通就回退
        pos, nxt = st.pop() 
        for i in range(nxt, 4): # 依次检查未探索方向
            nextp = (pos[0] + dirs[i][0], pos[1] + dirs[i][1])
            if nextp == end:
                print_path(end, pos, st) # 输出栈中的元素，当前位置和结束位置
                return
            if passable(maze, nextp): # 遇到未探查的新位置
                st.push((pos, i+1)) # 原位置和下一个要探查的方向先入栈
                mark(maze, nextp) # 标记已经找过了
                st.push((nextp, 0)) # 新位置也要入栈
                break # 跳出本次循环，下一次循环就是基于当前的新位置了
    
    print("path not found.")
    

In [57]:
maze = [[1,1,1,1,1],[1,0,0,1,1],[1,1,0,0,1],[1,1,1,0,1],[1,1,1,1,1]]
pos = (1,1)
end = (3,3)

In [58]:
maze_solve(maze, pos, end)

(2, 2) (1, 2) (1, 1) (2, 3) (3, 3)


### 基于队列的实现
基本框架：
```
将start标记为已达
start入队
while 队列中还有未充分探查的位置：
    取出一个位置pos
    检查pos的相邻位置
        遇到end成功结束
        尚未探查的都mark并入队
队列空，搜索失败
```
基于这个算法结构，定义相应的函数，基本结构与前面用栈的算法类似的队列作为缓存结构：

In [64]:
# 首先是SQueue的定义
class SQueue():
    def __init__(self, init_len = 8):
        self._len = init_len # 存储区长度
        self._elems = [0] * init_len # 元素存储
        self._head = 0 # 表头元素下标
        self._num = 0 # 元素个数
    
    def is_empty(self):
        return self._num == 0

    def peek(self):
        if self._num == 0:
            raise QueueUnderflow
        return self._elems[self._head]
    
    def dequeue(self):
        if self._num == 0:
            raise QueueUnderflow
        e = self._elems[self._head]
        self._head = (self._head + 1)%self._len
        self._num -= 1
        return e
    
    def enqueue(self, e):
        if self._num == self._len:
            self.__extend()
        self._elems[(self._head + self._num) % self._len] = e
        self._num += 1
        
    def __extend(self):
        old_len = self._len
        self._len *= 2
        new_elems = [0]*self._len
        for i in range(old_len):
            new_elems[i] = self._elems[(self._head + i) % old_len]
            self._elems, self._head = new_elems, 0

In [65]:
def maze_solver_queue(maze, start, end):
    if start == end: # 这是特殊情况
        print("Path finds.")
        return
    qu = SQueue()
    mark(maze, start)
    qu.enqueue(start) # 入口入队
    while not qu.is_empty(): # 还有候选位置
        pos = qu.dequeue() # 取出下一个位置
        for i in range(4):
            nextp = (pos[0] + dirs[i][0], pos[1] + dirs[i][1])
            if passable(maze, nextp):
                if nextp == end:
                    print("Path find.")
                    return
                mark(maze, nextp)
                qu.enqueue(nextp) # 新位置入队
    print("No path.")
        

In [66]:
maze = [[1,1,1,1,1],[1,0,0,1,1],[1,1,0,0,1],[1,1,1,0,1],[1,1,1,1,1]]
start = (1,1)
end = (3,3)
maze_solver_queue(maze, start, end)

Path find.


In [75]:
# 进阶的方法：打印出找到的路线
def maze_solver_queueV2(maze, start, end):
    if start == end: # 这是特殊情况
        print("Path finds.")
        return
    qu = SQueue()
    mark(maze, start)
    qu.enqueue(start) # 入口入队
    pos_dict = {}
    while not qu.is_empty(): # 还有候选位置
        pos = qu.dequeue() # 取出下一个位置
        for i in range(4):
            nextp = (pos[0] + dirs[i][0], pos[1] + dirs[i][1])
            if passable(maze, nextp):
                pos_dict[nextp] = pos # 记录位置的前驱位置
                if nextp == end:
                    print("Path find.The path is :", end = " ")
                    ppos = end
                    while ppos != start:
                        print(i, end = " ")
                        i = pos_dict[i]
                    print (start)
                    return
                mark(maze, nextp)
                qu.enqueue(nextp) # 新位置入队                
    print("No path.") # 没找到路线

In [74]:
maze = [[1,1,1,1,1],[1,0,0,1,1],[1,1,0,0,1],[1,1,1,0,1],[1,1,1,1,1]]
start = (1,1)
end = (3,3)
maze_solver_queueV2(maze, start, end)

Path find.The path is : (3, 3) (2, 3) (2, 2) (1, 2) (1, 1)
